Importing all libraries needed

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

2024-04-10 13:05:05.383129: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 13:05:14.589031: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 13:06:02.852815: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.__version__ 


'2.13.1'

Date Preprocessing
Generating images for the training set

In [3]:

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                #    rotation_range=40,
                                #    width_shift_range=0.2,
                                #    height_shift_range=0.2,
                                #    fill_mode='nearest'
                                   )

Generating images for the test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)

Creating the Training set

In [5]:
training_set = train_datagen.flow_from_directory('/home/alice/Desktop/Sign-Language-To-Text-Conversion/dataSet/trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 12453 images belonging to 27 classes.


In [9]:
testing_set = train_datagen.flow_from_directory('/home/alice/Desktop/Sign-Language-To-Text-Conversion/dataSet/testingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 4268 images belonging to 27 classes.


Building the CNN
Initializing the CNN

In [10]:
classifier = tf.keras.models.Sequential()

convolution

In [11]:

classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

pooling

In [12]:

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

Adding a second convolution layer

In [14]:

classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

flattening

In [15]:
classifier.add(tf.keras.layers.Flatten())

Full Connection

In [16]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

Training the CNN
compiling the CNN

In [18]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

Training the CNN on the Training set and evaluating it on the Test set

In [19]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 32)       

In [20]:
classifier.fit(training_set,
                  epochs = 5,
                  validation_data = testing_set)

Epoch 1/5


2024-04-10 13:14:54.875473: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.
2024-04-10 13:14:56.861249: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16777216 exceeds 10% of free system memory.
2024-04-10 13:14:56.892116: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16777216 exceeds 10% of free system memory.
2024-04-10 13:14:57.392762: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28311552 exceeds 10% of free system memory.
2024-04-10 13:14:57.705469: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16777216 exceeds 10% of free system memory.


390/390 [==============================] - 474s 1s/step - loss: 2.2428 - accuracy: 0.2912 - val_loss: 0.8066 - val_accuracy: 0.7610
Epoch 2/5
390/390 [==============================] - 407s 1s/step - loss: 0.9265 - accuracy: 0.6705 - val_loss: 0.2325 - val_accuracy: 0.9567
Epoch 3/5
390/390 [==============================] - 366s 938ms/step - loss: 0.5912 - accuracy: 0.7917 - val_loss: 0.0968 - val_accuracy: 0.9775
Epoch 4/5
390/390 [==============================] - 384s 983ms/step - loss: 0.4162 - accuracy: 0.8537 - val_loss: 0.0513 - val_accuracy: 0.9864
Epoch 5/5
390/390 [==============================] - 372s 953ms/step - loss: 0.3436 - accuracy: 0.8828 - val_loss: 0.0372 - val_accuracy: 0.9939


Saving the model

In [20]:
loss, accuracy = classifier.evaluate(testing_set)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

134/134 [==============================] - 33s 242ms/step - loss: 0.0603 - accuracy: 0.9906
Test Loss: 0.0603, Test Accuracy: 0.9906


In [21]:
model_json = classifier.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model1.h5')
print('Weights saved')

Model Saved
Weights saved
